# Blue Book for Bulldozers
#### Predict the auction sale price for a piece of heavy equipment to create a "blue book" for bulldozers.

The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.

Fast Iron is creating a "blue book for bull dozers," for customers to value what their heavy equipment fleet is worth at auction.

In [7]:
#imports
%load_ext autoreload
%autoreload 2
% matplotlib inline

In [8]:
# importing libraries
from fastai.imports import *
from fastai.structured import *

In [9]:
PATH = '/home/parth/Downloads/bulldozers/'

In [10]:
!ls {PATH}

Data Dictionary.xlsx		  Train.7z	     Train.zip
Machine_Appendix.csv		  TrainAndValid.7z   Valid.7z
median_benchmark.csv		  TrainAndValid.csv  Valid.csv
random_forest_benchmark_test.csv  TrainAndValid.zip  ValidSolution.csv
Test.csv			  Train.csv	     Valid.zip


In [17]:
#reading the dataframe
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

In [18]:
# checking the dataframe
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# converting the saleprice to the required asked value as given :-
df_raw['SalePrice']=np.log(df_raw['SalePrice'])

In [20]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# convert all string attributes to categorical attributes
train_cats(df_raw)

In [22]:
# extract useful information from Date and remove the column
add_datepart(df_raw,'saledate')

In [23]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200


In [32]:
# converting all categorical values to numerical values and applying one ot encoding and filling missing values
df,y,nas = proc_df(df_raw,'SalePrice',max_n_cat=6)

In [33]:
df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,fiModelDesc,fiBaseModel,fiSecondaryDesc,...,Differential_Type_Locking,Differential_Type_No Spin,Differential_Type_Standard,Differential_Type_nan,Steering_Controls_Command Control,Steering_Controls_Conventional,Steering_Controls_Four Wheel Standard,Steering_Controls_No,Steering_Controls_Wheel,Steering_Controls_nan
0,1139246,999089,3157,121,3.0,2004,68.0,950,296,41,...,0,0,1,0,0,1,0,0,0,0
1,1139248,117657,77,121,3.0,1996,4640.0,1725,527,55,...,0,0,1,0,0,1,0,0,0,0
2,1139249,434808,7009,121,3.0,2001,2838.0,331,110,0,...,0,0,0,1,0,0,0,0,0,1
3,1139251,1026470,332,121,3.0,2001,3486.0,3674,1375,0,...,0,0,0,1,0,0,0,0,0,1
4,1139253,1057373,17311,121,3.0,2007,722.0,4208,1529,0,...,0,0,0,1,0,0,0,0,0,1


In [31]:
print(y)

[11.09741 10.95081  9.21034 ...  9.3501   9.10498  8.95545]


In [75]:
# import sklearn 
from sklearn.ensemble import RandomForestRegressor

In [85]:
def split_vals(a,n):
    return a[:n].copy(),a[n:].copy()
   

In [86]:
n_valid=12_000
n_train=len(df)-n_valid

In [87]:
x_train,x_valid = split_vals(df,n_train)
y_train,y_valid = split_vals(y,n_train)

In [88]:
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_train), y_train), rmse(m.predict(x_valid), y_valid),
                m.score(x_train, y_train), m.score(x_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [89]:
type(x_train)

pandas.core.frame.DataFrame

Randomly changing hyper-parameters

In [94]:
m = RandomForestRegressor(n_estimators=10,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

CPU times: user 2min 25s, sys: 687 ms, total: 2min 26s
Wall time: 44.4 s
[0.0909612867188082, 0.25219949257849683, 0.9827078889302777, 0.8864109700005075]


In [95]:
m = RandomForestRegressor(n_estimators=20,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

CPU times: user 5min 9s, sys: 984 ms, total: 5min 10s
Wall time: 1min 23s
[0.08289841500730442, 0.25048761780769907, 0.9856375913252206, 0.8879477712851154]


## Using OOB score

In [96]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(x_train, y_train)
print_score(m)

[0.07867492216040756, 0.24157244118004306, 0.9870637777035696, 0.8957819962813351, 0.9079230613839963]


## Using SUBSAMPLING technique: to speed up and reduce overfitting

In [98]:
set_rf_samples=20_000
m=RandomForestRegressor(n_estimators=40,n_jobs=-1,oob_score=True)
m.fit(x_train,y_train)
print_score(m)

[0.07857029790511588, 0.24150408629268133, 0.9870981607741276, 0.8958409666009666, 0.9078005437186805]


In [99]:
m=RandomForestRegressor(n_estimators=40,max_features=0.5,min_samples_leaf=3,oob_score=True)
m.fit(x_train,y_train)
print_score(m)

[0.11975363243040535, 0.2322221556338104, 0.9700282420117895, 0.9036935713199286, 0.9105867080870066]


So the final validation score of 0.2322 was very good when compared to the leaderboard results on kaggle for the same competition.